In [1]:
import pandas as pd

In [2]:
orders_path  = '../project-2/orders.csv'
people_path  = '../project-2/people.csv'
returns_path = '../project-2/returns.csv'

In [3]:
orders  = pd.read_csv(orders_path)
people  = pd.read_csv(people_path)
returns = pd.read_csv(returns_path)

In [4]:
# 2. Preview
print("=== ORDERS ===")
display(orders.head())
print("\n=== PEOPLE ===")
display(people.head())
print("\n=== RETURNS ===")
display(returns.head())

=== ORDERS ===


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Customer Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2020-103800,2020-01-03,2020-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2020-141817,2020-01-05,2020-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840



=== PEOPLE ===


,Returned,Order ID
0,Yes,US-2020-100762
1,Yes,US-2020-100762
2,Yes,US-2020-100762
3,Yes,US-2020-100762
4,Yes,US-2020-100867



=== RETURNS ===


,Returned,Order ID
0,Yes,US-2020-100762
1,Yes,US-2020-100762
2,Yes,US-2020-100762
3,Yes,US-2020-100762
4,Yes,US-2020-100867


In [5]:
# 3. Basic info & null checks
print("\n--- Orders Info ---")
orders.info()
print("\n--- Orders Null Counts ---")
print(orders.isna().sum())


--- Orders Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Row ID            10194 non-null  int64  
 1   Order ID          10194 non-null  object 
 2   Order Date        10194 non-null  object 
 3   Ship Date         10194 non-null  object 
 4   Ship Mode         10194 non-null  object 
 5   Customer ID       10194 non-null  object 
 6   Customer Name     10194 non-null  object 
 7   Customer Segment  10194 non-null  object 
 8   Country/Region    10194 non-null  object 
 9   City              10194 non-null  object 
 10  State/Province    10194 non-null  object 
 11  Postal Code       10194 non-null  object 
 12  Region            10194 non-null  object 
 13  Product ID        10194 non-null  object 
 14  Category          10194 non-null  object 
 15  Sub-Category      10194 non-null  object 
 16  Product Name      1

In [6]:
# 4. Parse dates & numeric columns
orders['order_date'] = pd.to_datetime(orders['Order Date'], errors='coerce')
orders['ship_date']  = pd.to_datetime(orders['Ship Date'],  errors='coerce')
for col in ['Sales','Quantity','Discount','Profit']:
    orders[col.lower()] = pd.to_numeric(orders[col], errors='coerce')

In [7]:
# 5. Drop rows missing essential IDs & dedupe
orders_clean = orders.dropna(subset=['Order ID','Customer ID','Product ID']) \
                     .drop_duplicates()

In [8]:
# 6. Aggregate returns per Order ID
returns_count = (
    returns.groupby('Order ID')
           .size()
           .reset_index(name='returned_count')
)

In [9]:
# 7. Merge return counts back into orders
orders_clean = orders_clean.merge(
    returns_count, on='Order ID', how='left'
).fillna({'returned_count': 0})

In [11]:
orders_clean = orders_clean.rename(columns={
    'Order ID':          'order_id',
    'Customer ID':       'customer_id',
    'Customer Name':     'customer_name',
    'Customer Segment':  'customer_segment',
    'Ship Mode':         'ship_mode',
    'Country/Region':    'country_region',
    'City':              'city',
    'State/Province':    'state_province',
    'Postal Code':       'postal_code',
    'Region':            'region',
    'Product ID':        'product_id',
    'Product Name':      'product_name',
    'Category':          'category',
    'Sub-Category':      'sub_category'
})

In [12]:
keep_cols = [
    'order_id',
    'order_date',
    'ship_date',
    'ship_mode',
    'customer_id',
    'customer_name',
    'customer_segment',
    'country_region',
    'city',
    'state_province',
    'postal_code',
    'region',
    'product_id',
    'product_name',
    'category',
    'sub_category',
    'sales',
    'quantity',
    'discount',
    'profit',
    'returned_count'
]


In [13]:
orders_clean = orders_clean[keep_cols]

In [14]:
print(orders_clean.info())
print(orders_clean.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          10194 non-null  object        
 1   order_date        10194 non-null  datetime64[ns]
 2   ship_date         10194 non-null  datetime64[ns]
 3   ship_mode         10194 non-null  object        
 4   customer_id       10194 non-null  object        
 5   customer_name     10194 non-null  object        
 6   customer_segment  10194 non-null  object        
 7   country_region    10194 non-null  object        
 8   city              10194 non-null  object        
 9   state_province    10194 non-null  object        
 10  postal_code       10194 non-null  object        
 11  region            10194 non-null  object        
 12  product_id        10194 non-null  object        
 13  product_name      10194 non-null  object        
 14  category          1019

In [15]:
orders_clean.to_csv('orders_clean.csv', index=False)